In [ ]:
import time
import numpy as np
import datajoint as dj
import networkx as nx

# Visualizing the offline neuron

In [ ]:
import ipyvolume as ipv
import numpy as np
import trimesh
%matplotlib inline
ipv.figure()
# we draw the tetrahedron
neuron_data = np.load("107816118160698192_neuron_2_visualizations.npz")
original_mesh_verts = neuron_data["original_mesh_verts"]
original_mesh_faces = neuron_data["original_mesh_faces"]
edges = neuron_data["edges"]
unique_skeleton_verts = neuron_data["unique_skeleton_verts"]


original_mesh = trimesh.Trimesh(vertices=original_mesh_verts,faces=original_mesh_faces)


mesh = ipv.plot_trisurf(original_mesh.vertices[:,0], 
                        original_mesh.vertices[:,1],
                        original_mesh.vertices[:,2], 
                        triangles=original_mesh.faces, color='orange')

mesh.color = [0., 1., 0., 0.5]
mesh.material.transparent = True
# and also mark the vertices
mesh2 = ipv.plot_trisurf(unique_skeleton_verts[:,0], 
                        unique_skeleton_verts[:,1], 
                        unique_skeleton_verts[:,2], 
                        lines=edges, color='blue')

volume_maxs = np.max(original_mesh.vertices,axis=0)
volume_mins = np.min(original_mesh.vertices,axis=0)
ranges = volume_maxs - volume_mins
index = [0,1,2]
max_index = np.argmax(ranges)
min_limits = [0,0,0]
max_limits = [0,0,0]

buffer = 10000
for i in index:
    if i == max_index:
        min_limits[i] = volume_mins[i] - buffer
        max_limits[i] = volume_maxs[i] + buffer 
        continue
    else:
        difference = ranges[max_index] - ranges[i]
        min_limits[i] = volume_mins[i] - difference/2  - buffer
        max_limits[i] = volume_maxs[i] + difference/2 + buffer

#ipv.xyzlim(-2, 2)
ipv.xlim(min_limits[0],max_limits[0])
ipv.ylim(min_limits[1],max_limits[1])
ipv.zlim(min_limits[2],max_limits[2])
ipv.show()

In [ ]:
# Example of how to put a skeleton into a networkx mdoel
print(unique_skeleton_verts)
edges

In [ ]:
B = nx.Graph()
B.add_nodes_from([(x,{"coordinates":y}) for x,y in enumerate(unique_skeleton_verts)])
B.add_edges_from(edges_with_coefficients)

#now calculate the weights of the edge from 

In [ ]:
# how to create it from adjacency matrix
# G=nx.from_numpy_matrix(A)

In [ ]:
G = nx.Graph()
G.add_edges_from(original_mesh.face_adjacency)

In [ ]:
original_mesh.face_adjacency

In [ ]:
edges_as_vertices = original_mesh.vertices[original_mesh.edges]
distances = np.linalg.norm(edges_as_vertices[:,1]  - edges_as_vertices[:,0]
                           ,axis=1)
distances.shape

In [ ]:
G = nx.Graph()

In [ ]:
G.add_weighted_edges_from

# Shortest Path algorithm from trimesh example:

In [10]:
import trimesh
file_1 = "107816118160698192_top_apical_axon_length.off"
file_2 = "107816118160698192_top_apical_smaller.off"
file_3 = "107816118160698192_top_apical.off"
current_mesh = trimesh.load_mesh(file_3)

In [11]:
mesh_lists = current_mesh.split(only_watertight = False)
import networkx as nx
import time 
import numpy as np

start_time = time.time()

# test on a sphere mesh
mesh = mesh_lists[0]

In [12]:
g = nx.Graph()
ga = nx.from_edgelist(mesh.edges)

In [13]:
# # edges without duplication
# edges = mesh.edges_unique

# # the actual length of each unique edge
# length = mesh.edges_unique_length

# # create the graph with edge attributes for length
# g = nx.Graph()
# # for edge, L in zip(edges, length):
# #     g.add_edge(*edge, length=L)

# # alternative method for weighted graph creation
# # you can also create the graph with from_edgelist and
# # a list comprehension, which is like 1.5x faster
# ga = nx.from_edgelist([(e[0], e[1], {'length': L})
#                        for e, L in zip(edges, length)])

# #nx.diameter(ga)

# print("Total time for diameter = " + str(start_time - time.time()))

# # # arbitrary indices of mesh.vertices to test with
# # start = 0
# # end = int(len(mesh.vertices) / 2.0)

# # # run the shortest path query using length for edge weight
# # path = nx.shortest_path(g,
# #                         weight='length')

# # # VISUALIZE RESULT
# # # make the sphere transparent-ish
# # mesh.visual.face_colors = [100, 100, 100, 100]
# # # Path3D with the path between the points
# # path_visual = trimesh.load_path(mesh.vertices[path])
# # # visualizable two points
# # points_visual = trimesh.points.PointCloud(mesh.vertices[[start, end]])

# # # create a scene with the mesh, path, and points
# # scene = trimesh.Scene([
# #     points_visual,
# #     path_visual,
# #     mesh])

# # scene.show(smooth=False)

In [14]:
# import random
# from tqdm import tqdm
# k = 500
# sampled_nodes = random.sample(ga.nodes, k)

# lp_start = -1
# lp_end = -1
# lp_magnitude = -1

# for s in tqdm(sampled_nodes): 
#     sp_dict = nx.single_source_shortest_path_length(ga,s,weight='length')
    
#     list_keys = list(sp_dict.keys())
#     longest_path_node = list_keys[len(list_keys)-1]
#     longest_path_magnitude = sp_dict[longest_path_node]
    
#     if longest_path_magnitude > lp_magnitude:
#         lp_start = s
#         lp_end = longest_path_node
#         lp_magnitude = longest_path_magnitude

In [15]:
import random
from tqdm import tqdm
k = 500
sampled_nodes = random.sample(ga.nodes, k)


lp_end_list = []
lp_magnitude_list = []

for s in tqdm(sampled_nodes): 
    sp_dict = nx.single_source_shortest_path_length(ga,s)
    
    list_keys = list(sp_dict.keys())
    longest_path_node = list_keys[len(list_keys)-1]
    longest_path_magnitude = sp_dict[longest_path_node]
    

    lp_end_list.append(longest_path_node)
    lp_magnitude_list.append(longest_path_magnitude)

100%|██████████| 500/500 [01:45<00:00,  4.75it/s]


In [16]:
#construct skeleton from shortest path
import numpy as np
final_start = sampled_nodes[np.argmax(lp_magnitude_list)]
final_end = sampled_nodes[np.argmax(lp_end_list)]

node_list = nx.shortest_path(ga,final_start,final_end)
skeleton_vertices = mesh.vertices[node_list]
#skeleton_edges = np.vstack([node_list[:-1],node_list[1:]]).T
skeleton_vertices_indexes = np.arange(len(skeleton_vertices))
skeleton_edges = np.vstack([skeleton_vertices_indexes[:-1],
                           skeleton_vertices_indexes[1:]]).T


# graphing the mesh and the skeleton edges

In [19]:
skeleton_edges

array([[  0,   1],
       [  1,   2],
       [  2,   3],
       [  3,   4],
       [  4,   5],
       [  5,   6],
       [  6,   7],
       [  7,   8],
       [  8,   9],
       [  9,  10],
       [ 10,  11],
       [ 11,  12],
       [ 12,  13],
       [ 13,  14],
       [ 14,  15],
       [ 15,  16],
       [ 16,  17],
       [ 17,  18],
       [ 18,  19],
       [ 19,  20],
       [ 20,  21],
       [ 21,  22],
       [ 22,  23],
       [ 23,  24],
       [ 24,  25],
       [ 25,  26],
       [ 26,  27],
       [ 27,  28],
       [ 28,  29],
       [ 29,  30],
       [ 30,  31],
       [ 31,  32],
       [ 32,  33],
       [ 33,  34],
       [ 34,  35],
       [ 35,  36],
       [ 36,  37],
       [ 37,  38],
       [ 38,  39],
       [ 39,  40],
       [ 40,  41],
       [ 41,  42],
       [ 42,  43],
       [ 43,  44],
       [ 44,  45],
       [ 45,  46],
       [ 46,  47],
       [ 47,  48],
       [ 48,  49],
       [ 49,  50],
       [ 50,  51],
       [ 51,  52],
       [ 52,

In [20]:
np.savez("../apial_mesh_skeleton.npz",mesh=current_mesh,skeleton_edges=edges,
        skeleton_vertices=skeleton_vertices)

In [18]:
import ipyvolume as ipv
import numpy as np
import trimesh
%matplotlib inline
ipv.figure()
# we draw the tetrahedron

current_mesh = mesh
original_mesh_verts = current_mesh.vertices
original_mesh_faces = current_mesh.faces
edges = skeleton_edges
unique_skeleton_verts = skeleton_vertices


original_mesh = trimesh.Trimesh(vertices=original_mesh_verts,faces=original_mesh_faces)


mesh = ipv.plot_trisurf(original_mesh.vertices[:,0], 
                        original_mesh.vertices[:,1],
                        original_mesh.vertices[:,2], 
                        triangles=original_mesh.faces, color='orange')

mesh.color = [0., 1., 0., 0.2]
mesh.material.transparent = True
# and also mark the vertices
mesh2 = ipv.plot_trisurf(unique_skeleton_verts[:,0], 
                        unique_skeleton_verts[:,1], 
                        unique_skeleton_verts[:,2], 
                        lines=edges, color='blue')

volume_maxs = np.max(original_mesh.vertices,axis=0)
volume_mins = np.min(original_mesh.vertices,axis=0)
ranges = volume_maxs - volume_mins
index = [0,1,2]
max_index = np.argmax(ranges)
min_limits = [0,0,0]
max_limits = [0,0,0]

buffer = 10000
for i in index:
    if i == max_index:
        min_limits[i] = volume_mins[i] - buffer
        max_limits[i] = volume_maxs[i] + buffer 
        continue
    else:
        difference = ranges[max_index] - ranges[i]
        min_limits[i] = volume_mins[i] - difference/2  - buffer
        max_limits[i] = volume_maxs[i] + difference/2 + buffer

#ipv.xyzlim(-2, 2)
ipv.xlim(min_limits[0],max_limits[0])
ipv.ylim(min_limits[1],max_limits[1])
ipv.zlim(min_limits[2],max_limits[2])
ipv.show()

In [15]:
import trimesh
current_mesh = trimesh.load_mesh(file_1)

In [16]:
import ipyvolume as ipv
import numpy as np
import trimesh
%matplotlib inline
ipv.figure()
# we draw the tetrahedron

original_mesh_verts = current_mesh.vertices
original_mesh_faces = current_mesh.faces
edges = skeleton_edges
unique_skeleton_verts = skeleton_vertices


original_mesh = trimesh.Trimesh(vertices=original_mesh_verts,faces=original_mesh_faces)


mesh = ipv.plot_trisurf(original_mesh.vertices[:,0], 
                        original_mesh.vertices[:,1],
                        original_mesh.vertices[:,2], 
                        triangles=original_mesh.faces, color='orange')

mesh.color = [0., 1., 0., 0.5]
mesh.material.transparent = True
# and also mark the vertices
# mesh2 = ipv.plot_trisurf(unique_skeleton_verts[:,0], 
#                         unique_skeleton_verts[:,1], 
#                         unique_skeleton_verts[:,2], 
#                         lines=edges, color='blue')

volume_maxs = np.max(original_mesh.vertices,axis=0)
volume_mins = np.min(original_mesh.vertices,axis=0)
ranges = volume_maxs - volume_mins
index = [0,1,2]
max_index = np.argmax(ranges)
min_limits = [0,0,0]
max_limits = [0,0,0]

buffer = 10000
for i in index:
    if i == max_index:
        min_limits[i] = volume_mins[i] - buffer
        max_limits[i] = volume_maxs[i] + buffer 
        continue
    else:
        difference = ranges[max_index] - ranges[i]
        min_limits[i] = volume_mins[i] - difference/2  - buffer
        max_limits[i] = volume_maxs[i] + difference/2 + buffer

#ipv.xyzlim(-2, 2)
ipv.xlim(min_limits[0],max_limits[0])
ipv.ylim(min_limits[1],max_limits[1])
ipv.zlim(min_limits[2],max_limits[2])
ipv.show()

In [ ]:
current_mesh.show()